# Instalando o Spark

In [1]:
# !pip install pyspark #==3.3.1

In [3]:
""" !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip """

' !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip ngrok-stable-linux-amd64.zip '

# Iniciando Sessão Spark

In [7]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import year, month, col 

In [1]:

# ConfigureSparkUI
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder                  # Método da classe que constrói a sessão spark
      .appName("Exercicios Spark")  # Nome do App Spark
      .getOrCreate())                     # Verifica se há uma sessão ativa, e se não há, cria uma nova sessão

23/10/14 15:41:57 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.54 instead (on interface enp3s0)
23/10/14 15:41:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/14 15:42:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
get_ipython().system_raw('./ngrok http 4050 &')

In [29]:
!curl -s http://localhost:4040/api/tunnels 

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://3b9d-2804-88c-7f3d-ea00-85b5-6195-28e-4054.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://3b9d-2804-88c-7f3d-ea00-85b5-6195-28e-4054.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [4]:
get_ipython().system_raw('./ngrok config add-authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT')


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

caminho_csv = "./base_de_dados.csv"

df = spark.read.csv(
      path=caminho_csv,
      header=True,
      sep=";",
      schema=schema_base_pix,
      timestampFormat="dd/MM/yyyy HH:mm"
)

# Exercícios
1. Descubra a quantidade total de transações na base de dados.
2. Qual foi a quantia total de dinheiro transacional em cada ano?
2. Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.
2. Retorne a quantidade de transações agrupadas por tipo de chave.
3. Descubra qual é a média, mínima e máxima do valor de todas as transações.
4. Quais foram as 5 maiores transações durante todo o período?
5. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações. 
6. Descubra qual foi a maior transação de cada mês na base de dados.
7. Crie uma nova coluna que contenha o valor da transação em dólar. 

In [ ]:
df.show()

In [6]:
# Descubra a quantidade total de transações na base de dados.

df.count()

100

In [9]:



novo_df = df.withColumn('ano', year(col('data_transacao')))



In [11]:
# Qual foi a quantia total de dinheiro transacional em cada ano?

novo_df.groupBy('ano').sum('valor').show()

+----+------------------+
| ano|        sum(valor)|
+----+------------------+
|2022|517980.58999999973|
|2021| 848320.6000000001|
+----+------------------+



In [14]:
# Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.

novo_df.filter(col('parte_creditada_banco')=='Nubank').groupBy('parte_creditada_banco').sum('valor').show()

+---------------------+----------+
|parte_creditada_banco|sum(valor)|
+---------------------+----------+
|               Nubank| 302224.64|
+---------------------+----------+



In [14]:
# Retorne a quantidade de transações agrupadas por tipo de chave.

novo_df.groupBy('chave_pix_tipo').count().show()    

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|       celular|   22|
|         email|   29|
|           cpf|   49|
+--------------+-----+



In [ ]:
# Descubra qual é a média, mínima e máxima do valor de todas as transações.


In [15]:
# novo_df.agg({'valor': 'min'}).show()
# novo_df.agg({ 'valor': 'max' }).show()
# novo_df.agg({ 'valor': 'mean' }).show()

novo_df.select('valor').summary('mean', 'min', 'max').show()

+-------+------------------+
|summary|             valor|
+-------+------------------+
|   mean|13663.011899999998|
|    min|              0.03|
|    max|          95977.62|
+-------+------------------+



In [25]:
 # Quais foram as 5 maiores transações durante todo o período?
novo_df.select(col('parte_debitada_nome'),col('valor')).orderBy('valor', ascending=False).show(5)


+--------------------+--------+
| parte_debitada_nome|   valor|
+--------------------+--------+
|   Henrique Ferreira|95977.62|
|Joao Vitor Cavalc...|94736.79|
|      Henrique Pinto|94586.45|
|      Giovanna Costa|81977.98|
|      Eduarda Campos|80083.34|
+--------------------+--------+
only showing top 5 rows



In [19]:
# Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações. 

novo_df.filter(
    (col('parte_debitada_banco')=='BTG' ) | 
    (col('parte_creditada_banco')=='BTG')
).groupBy('chave_pix_tipo').count().show()


+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|       celular|   13|
|         email|   15|
|           cpf|   22|
+--------------+-----+



In [28]:
# Descubra qual foi a maior transação de cada mês na base de dados.

novo_df.filter(
    (year(col('data_transacao'))==2021) 
).groupBy(month('data_transacao')).max('valor').show()

novo_df.filter(
    (year(col('data_transacao'))==2022)
).groupBy(month('data_transacao')).max('valor').show()

+---------------------+----------+
|month(data_transacao)|max(valor)|
+---------------------+----------+
|                   12|    829.87|
|                    1|  94736.79|
|                    6|  60139.23|
|                    3|  78347.58|
|                    5|    611.53|
|                    9|  95977.62|
|                    4|     459.2|
|                    8|    570.52|
|                    7|   9320.87|
|                   10|   1508.83|
|                   11|  80083.34|
|                    2|  81977.98|
+---------------------+----------+



+---------------------+----------+
|month(data_transacao)|max(valor)|
+---------------------+----------+
|                   12|  57433.69|
|                    1|   78559.4|
|                    6|  48714.95|
|                    3|  35409.61|
|                    5|  38219.08|
|                    9|  33629.97|
|                    4|  20875.64|
|                    8|   4374.56|
|                    7|  49836.72|
|                   10|   2842.73|
|                   11|    890.47|
|                    2|   7944.02|
+---------------------+----------+



In [13]:
# Crie uma nova coluna que contenha o valor da transação em dólar.

df_dolar = novo_df.withColumn('dolar', col('valor') / 5.10)
df_dolar.show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+----+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao| ano|              dolar|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+----+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|2022|  1.947058823529412|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|    